In [1]:
import os
import sys
sys.path.append(os.path.abspath('../'))
os.chdir(os.path.abspath('../'))
from scripts.eval_phee.sample_all import get_label_mapper, read_gold_result, read_pred_results 
import random
import tabulate
from IPython.display import display_html

In [2]:
from chatgpt_few_shot.prompter import Prompter

/home/u2081275/anaconda3/envs/uie/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (5.1.0)/charset_normalizer (2.0.10) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [3]:
m1 = "BM25-Type"
m2 = "TreeKernelW-Type"
demo_num = 5

data_folder = "data/converted_data/text2spotasoc/event/phee_2/"
# config_file = "dataset_processing/data_config/event/phee2.yaml"
# gold_file = "data/datasets/phee2/source/test.json"
instruction_template = "chatgpt_few_shot/instruction_templates/explanation.json"
query_template = "chatgpt_few_shot/query_templates/explanation.json"

prompter1 = Prompter(data_folder, demo_num=demo_num, demo_strategy=m1, instruction_template=instruction_template, query_template=query_template)
prompter2 = Prompter(data_folder, demo_num=demo_num, demo_strategy=m2, instruction_template=instruction_template, query_template=query_template)

case_list = prompter1.test_list

lb_mapper = get_label_mapper(config_file)
gold_instances = read_gold_result(gold_file)

assert(len(case_list) == len(gold_instances))

In [39]:
while True:
    sample_case = random.sample(case_list, 1)[0]
    test_instance = prompter1.get_an_instance(sample_case)
    test_sent = test_instance["sentence"]

    if test_instance['event_types'][0] != 'potential therapeutic event':
        continue

    break


import json

def read_pred_instance(model, test_id):
    events = []
    rst_path = "chatgpt_few_shot/output/%s_demo_5/test/%s.json"%(model, test_id)
    with open(rst_path, 'r') as f:
        data = json.load(f)
        for event in json.loads(data["answer"]):
            etp = event["event_type"]
            arguments = event["arguments"]
            arguments["event_type"] = etp
            events.append(arguments)
    return events
    
pred1 = read_pred_instance(m1, sample_case)
pred2 = read_pred_instance(m2, sample_case)

print(test_instance['sentence'])
table = []
table.append(["", "gold", m1, m2])
for eid, event in enumerate(json.loads(test_instance['answer'])):
    if eid < len(pred1):
        event1 = pred1[eid]
    else:
        event1 = {}
    if eid < len(pred2):
        event2 = pred2[eid]
    else:
        event2 = {}

    event["arguments"]["event_type"] = event["event_type"]
    for k, v in event["arguments"].items():
        arg_line = [k, v]
        if k in event1:
            arg_line.append(event1[k])
        else:
            arg_line.append("")
        if k in event2:
            arg_line.append(event2[k])
        else:
            arg_line.append("")
        table.append(arg_line)

tabulate.tabulate(table, tablefmt='html')

Treatment of a patient with severe digitoxin intoxication by Fab fragments of anti - digitalis antibodies .


,gold,BM25-Type,TreeKernelW-Type
subject,a patient with severe digitoxin intoxication,a patient with severe digitoxin intoxication,a patient with severe digitoxin intoxication
treatment,Fab fragments of anti - digitalis antibodies,Fab fragments of anti - digitalis antibodies,Fab fragments of anti - digitalis antibodies
effect,N/A,N/A,N/A
age,N/A,N/A,N/A
gender,N/A,N/A,N/A
race,N/A,N/A,N/A
population,N/A,N/A,N/A
subject_disorder,digitoxin intoxication,severe digitoxin intoxication,severe digitoxin intoxication
drug,Fab fragments of anti - digitalis antibodies,Fab fragments of anti - digitalis antibodies,Fab fragments of anti - digitalis antibodies
dosage,N/A,N/A,N/A


In [40]:
for prompter, strategy in zip([prompter1, prompter2], [m1, m2]):
    print("Query:", test_sent)
    retrievals = prompter.get_demostrations_prompt(test_instance)
    print("%s Retrivals:"%strategy)
    for sample in retrievals:
        if sample['role'] == 'user':
            print(sample['content'].replace(" Output:",""))
    print("")

Query: Treatment of a patient with severe digitoxin intoxication by Fab fragments of anti - digitalis antibodies .
BM25-Type Retrivals:
Sentence: Treatment with infliximab is known to produce an increase of autoantibodies ( antinuclear antibodies , anti - double - stranded DNA ) , but not clinical disease .
Sentence: Neurointensive care management of raised intracranial pressure caused by severe valproic acid intoxication .
Sentence: Severe water intoxication secondary to the concomitant intake of non - steroidal anti - inflammatory drugs and desmopressin : a case report and review of the literature .
Sentence: Treatment of chronic hepatitis C with interferon alpha ( IFN - alpha ) is relatively contraindicated in patients with psychiatric disorders because of possible severe psychiatric side effects .
Sentence: A patient with alcoholic cerebellar degeneration had periodic alternating nystagmus during a period of phenytoin intoxication .
Sentence: Flumazenil reversal of benzodiazepine -